In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install Pillow

In [ ]:
pip install rembg

Remove background

In [ ]:
from rembg import remove
from PIL import Image
from PIL import ImageOps
import os

input_directory = '/content/drive/MyDrive/input2/'
output_directory = '/content/drive/MyDrive/outputimages/'


os.makedirs(output_directory, exist_ok=True)

for filename in os.listdir(input_directory):
    if filename.endswith(".PNG") or filename.endswith(".jpg") or filename.endswith(".png"):
        input_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, filename)

        input_image = Image.open(input_path)

        output = remove(input_image, alpha_matting=True)

        background_color = (255, 255, 255)
        new_image = Image.new("RGB", output.size, background_color)
        new_image.paste(output, (0, 0), output)
        new_image.save(output_path, format='PNG')

print("Processing complete")

Add border and resize

In [ ]:
import os
from PIL import Image

input_folder = '/content/drive/MyDrive/outputimages/'
output_folder = '/content/drive/MyDrive/output2/'

min_padding = 70
target_size = (1920, 2400)
i=0

image_paths = []
for root, dirs, files in os.walk(input_folder):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.png')):
            image_path = os.path.join(root, filename)
            image_paths.append(image_path)

for image_path in image_paths:
    image = Image.open(image_path)


    left_distance = 0
    top_distance = 0
    right_distance = 0
    bottom_distance = 0

    for x in range(image.width):
        if all(image.getpixel((x, y)) == (255, 255, 255) for y in range(image.height)):
            left_distance += 1
        else:
            break

    for x in range(image.width - 1, -1, -1):
        if all(image.getpixel((x, y)) == (255, 255, 255) for y in range(image.height)):
            right_distance += 1
        else:
            break

    for y in range(image.height):
        if all(image.getpixel((x, y)) == (255, 255, 255) for x in range(image.width)):
            top_distance += 1
        else:
            break

    for y in range(image.height - 1, -1, -1):
        if all(image.getpixel((x, y)) == (255, 255, 255) for x in range(image.width)):
            bottom_distance += 1
        else:
            break

    padding_x = max(min_padding - left_distance, 0) + max(min_padding - right_distance, 0)
    padding_y = max(min_padding - top_distance, 0) + max(min_padding - bottom_distance, 0)

    new_width = image.width + padding_x
    new_height = image.height + padding_y

    new_image = Image.new("RGB", (new_width, new_height), (255, 255, 255))

    paste_x = max(min_padding - left_distance, 0)
    paste_y = max(min_padding - top_distance, 0)
    new_image.paste(image, (paste_x, paste_y))

    new_image = new_image.resize(target_size, Image.ANTIALIAS)


    relative_path = os.path.relpath(image_path, input_folder)
    output_path = os.path.join(output_folder, relative_path)


    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    new_image.save(output_path)
    i=i+1
    print("Completion of image processing:",i)


Resize only

In [ ]:
from PIL import Image
import os

input_folder1 = '/content/drive/MyDrive/outputimages/'
output_folder = '/content/drive/MyDrive/output2/'
target_size = (1920, 2400)


image_paths = []
for root, dirs, files in os.walk(input_folder1):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.png')):
            image_path = os.path.join(root, filename)
            image_paths.append(image_path)

for image_path in image_paths:
    image = Image.open(image_path)

    new_image = image.resize(target_size, Image.ANTIALIAS)


    relative_path = os.path.relpath(image_path, input_folder1)
    output_path = os.path.join(output_folder, relative_path)


    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    new_image.save(output_path)


Put object of image to central of image background

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def process_image(image_file):
    image_path = os.path.join(input_folder, image_file)
    original_image = Image.open(image_path)

    object_mask = Image.new('L', original_image.size)

    for x in range(original_image.width):
        for y in range(original_image.height):
            pixel = original_image.getpixel((x, y))
            if pixel != white_color:
                object_mask.putpixel((x, y), 255)
            else:
                object_mask.putpixel((x, y), 0)

    bbox = object_mask.getbbox()

    object_image = original_image.crop(bbox)

    new_background = Image.new('RGB', (new_width, new_height), white_color)

    x_offset = (new_width - object_image.width) // 2
    y_offset = (new_height - object_image.height) // 2

    new_background.paste(object_image, (x_offset, y_offset))

    output_path = os.path.join(output_folder, image_file)
    new_background.save(output_path)

input_folder = '/content/drive/MyDrive/output2/'
output_folder = '/content/drive/MyDrive/output2_processed/'

os.makedirs(output_folder, exist_ok=True)

new_width = 1920
new_height = 2400
white_color = (255, 255, 255)

image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.png'))]

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    executor.map(process_image, image_files)